In [1]:
import pdfplumber as pdf
import pandas as pd
from polyfuzz import PolyFuzz
from polyfuzz.models import SentenceEmbeddings
from sentence_transformers import SentenceTransformer

/Users/Larry/data-cleaning/env3.8/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-10-28 18:31:06.393124: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
pdf_tw = pdf.open("./pdf/tw.pdf")
pdf_eu = pdf.open("./pdf/35to129_eu.pdf")

### Taiwan


In [4]:
tw_pdf = []
with pdf_tw as pdf:
    pages = pdf.pages
    for page in pages:
        page = page.extract_table()
        for table in page:
            tw_pdf.append(table)

tw_pdf = pd.DataFrame(data=tw_pdf[1:-1], columns=tw_pdf[0]).drop(["No."], axis=1)
tw_pdf = tw_pdf.drop_duplicates(keep=False)
tw_pdf = tw_pdf.reset_index(drop=True)

tw_pdf.replace(to_replace="-", value="NO CAS", regex=False, inplace=True)
tw_pdf.rename(columns={"Chemical name": "Chemical Name"}, inplace=True)

# tw_pdf = tw_pdf.assign(**{"CAS No.": tw_pdf["CAS No."].str.split("\n")})

# Explode the "CAS No." column to create separate rows for each CAS number
# tw_pdf = tw_pdf.explode("CAS No.")

# Remove leading/trailing whitespace from the "CAS No." column
# tw_pdf["CAS No."] = tw_pdf["CAS No."].str.strip()

# Drop duplicates
# tw_pdf = tw_pdf.drop_duplicates()

# tw_pdf["CAS No."] = tw_pdf["CAS No."].replace(to_replace="/", value="", regex=True)

# Reset the index
tw_pdf = tw_pdf.reset_index(drop=True)

tw_pdf

,Chemical Name,CAS No.
0,Mercury and its compounds (with the exception ...,7439-97-6
1,4-Benzyloxyphenol and 4-ethoxyphenol,103-16-2/\n622-62-8
2,Bithionol,97-18-7
3,Pilocarpine and its salts,92-13-7
4,Halogeno-salicylanilide,NO CAS
...,...,...
646,Raw material made from bovine and sheep tissue...,NO CAS
647,Alanroot oil（Inula helenium L.）,97676-35-2
648,"Rauwolfia serpentina L., alkaloids and their s...",90106-13-1
649,Yohimbine and its salts,146-48-5


### EU


In [5]:
eu_pdf = []
with pdf_eu as pdf:
    pages = pdf.pages
    for page in pages:
        page = page.extract_table()
        for table in page:
            eu_pdf.append(table)
eu_pdf = pd.DataFrame(data=eu_pdf[1:], columns=eu_pdf[0])
eu_pdf = eu_pdf.drop_duplicates(keep=False).reset_index(drop=True)

In [6]:
eu_pdf.rename(
    columns={"Chemical name/INN": "Chemical Name", "CAS number": "CAS No."},
    inplace=True,
)

# on9space = eu_pdf["CAS No."].iloc[15]

# eu_pdf["CAS No."] = eu_pdf["CAS No."].replace(
#    to_replace=on9space, value="NO CAS", regex=False
# )

# eu_pdf = eu_pdf.assign(**{"CAS No.": eu_pdf["CAS No."].str.split("\n")})

# Explode the "CAS No." column to create separate rows for each CAS number
# eu_pdf = eu_pdf.explode("CAS No.")

# Remove leading/trailing whitespace from the "CAS No." column
# eu_pdf["CAS No."] = eu_pdf["CAS No."].str.strip()

# Drop duplicates
# eu_pdf = eu_pdf.drop_duplicates()

# eu_pdf["CAS No."] = eu_pdf["CAS No."].replace(to_replace="/", value="", regex=True)
# eu_pdf.replace(to_replace="—", value="NO CAS", regex=False, inplace=True)
# eu_pdf.replace(to_replace="\n", value=" ", regex=True, inplace=True)
# eu_pdf["CAS No."] = eu_pdf["CAS No."].replace("\\[[^\\]]*\\]", "", regex=True)

# another7space = eu_pdf["CAS No."].iloc[1868]
# eu_pdf["CAS No."] = eu_pdf["CAS No."].replace(another7space, "NO CAS", regex=True)

# Reset the index
eu_pdf = eu_pdf.reset_index(drop=True)

eu_pdf

,Chemical Name,CAS No.
0,N-(5-Chlorobenzoxazol-2-yl)acetamide,35783-57-4
1,(2-Acetoxyethyl)trimethylammonium hydroxide\n(...,51-84-3
2,Deanol aceglumate (INN),3342-61-8
3,Spironolactone (INN),52-01-7
4,"[4-(4-Hydroxy-3-iodophenoxy)-3,5-diiodophenyl]...",51-24-1
...,...,...
1579,Reaction products of paraformaldehyde with 2-\...,—
1580,Methylhydrazine,60-34-4
1581,"Triadimenol (ISO); (1RS,2RS;1RS,2SR)-1-(4-chlo...",55219-65-3
1582,Thiacloprid (ISO); (Z)-3-(6-chloro-3-pyridyl-m...,111988-49-9


Utilizing Polyfuzz to fuzzy match chemical names

NLP very cool


Extract chemical names columns from the dataframes as list:


In [7]:
print(len(tw_pdf), len(eu_pdf))
eu_names = eu_pdf.drop_duplicates(subset=["Chemical Name"], keep="first")
tw_names = tw_pdf.drop_duplicates(subset=["Chemical Name"], keep="first")
eu_names = eu_names["Chemical Name"].tolist()
tw_names = tw_names["Chemical Name"].tolist()


# Naming them as "to" and "from" to avoid confusion during matching
from_list = eu_names
to_list = tw_names
print(len(from_list), len(to_list))

651 1584
1582 651


In [9]:
threshold = 0.75

In [35]:

distance_model = SentenceEmbeddings(
    "all-mpnet-base-v2",
    min_similarity=threshold,
    cosine_method="sklearn",
)
model = PolyFuzz(distance_model).match(from_list, to_list)

In [36]:
result = model.get_matches().sort_values(by=["Similarity"], ascending=False)
result.rename(columns={"From": "EU", "To": "TW"}, inplace=True)

tw_common = result["TW"]
eu_common = result["EU"]

In [37]:
import openpyxl

with pd.ExcelWriter("./result/nlp_match_v2.xlsx") as writer:
    result.to_excel(writer, sheet_name="Simiarlity", index=False)

Embeddings


1. eu + tw then dedupe, merge to existing eu list

2. for each item in tw list, fuzzy match onto eu list??

### Comparison

on chemical names


In [81]:
comparison = pd.merge(
    tw_pdf["Chemical Name"], eu_pdf["Chemical Name"], how="outer", indicator="Exist"
)
comparison.replace(to_replace="left_only", value="Taiwan", inplace=True)
comparison.replace(to_replace="right_only", value="EU", inplace=True)
comparison.replace(to_replace="both", value="Both", inplace=True)
comparison

,Chemical Name,Exist
0,Mercury and its compounds (with the exception ...,Taiwan
1,4-Benzyloxyphenol and 4-ethoxyphenol,Both
2,4-Benzyloxyphenol and 4-ethoxyphenol,Both
3,4-Benzyloxyphenol and 4-ethoxyphenol,Both
4,4-Benzyloxyphenol and 4-ethoxyphenol,Both
...,...,...
2304,Reaction products of paraformaldehyde with 2- ...,EU
2305,Methylhydrazine,EU
2306,"Triadimenol (ISO); (1RS,2RS;1RS,2SR)-1-(4-chlo...",EU
2307,Thiacloprid (ISO); (Z)-3-(6-chloro-3-pyridyl-m...,EU


on CAS no.


In [111]:
cas_comp = tw_pdf.merge(
    eu_pdf,
    on="CAS No.",
    how="outer",
    indicator="CAS Exist",
    suffixes=("(Taiwan)", "(EU)"),
)
cas_comp.replace(to_replace="left_only", value="Taiwan", inplace=True)
cas_comp.replace(to_replace="right_only", value="EU", inplace=True)
cas_comp.replace(to_replace="both", value="Both", inplace=True)
cas_comp

,Chemical Name(Taiwan),CAS No.,Chemical Name(EU),CAS Exist
0,Mercury and its compounds (with the exception ...,7439-97-6,"Mercury and its compounds, except those specia...",Both
1,4-Benzyloxyphenol and 4-ethoxyphenol,103-16-2,4-Benzyloxyphenol and 4-ethoxyphenol,Both
2,4-Benzyloxyphenol and 4-ethoxyphenol,622-62-8,4-Benzyloxyphenol and 4-ethoxyphenol,Both
3,Bithionol,97-18-7,Bithionol (INN),Both
4,Pilocarpine and its salts,92-13-7,Pilocarpine and its salts,Both
...,...,...,...,...
7634,NaN,3830-45-3,Nonadecafluorodecanoic acid [1] Ammonium nonad...,EU
7635,NaN,5625-90-1,"N,N′-Methylenedimorpholine; N,N′-methylenebism...",EU
7636,NaN,55219-65-3,"Triadimenol (ISO); (1RS,2RS;1RS,2SR)-1-(4-chlo...",EU
7637,NaN,111988-49-9,Thiacloprid (ISO); (Z)-3-(6-chloro-3-pyridyl-m...,EU


In [112]:
cas_comp.drop(["CAS Exist"], axis=1, inplace=True)
cas_comp.reset_index(drop=True, inplace=True)
cas_comp.to_csv("./result/cas_comp.csv", index=False)

In [101]:
no_cas_excl = cas_comp.query("`CAS No.` != 'NO CAS'")
all_no_cas = cas_comp.query("`CAS No.` == 'NO CAS'")
no_cas_excl

,Chemical Name(Taiwan),CAS No.,Chemical Name(EU),CAS Exist
0,Mercury and its compounds (with the exception ...,7439-97-6,"Mercury and its compounds, except those specia...",Both
1,4-Benzyloxyphenol and 4-ethoxyphenol,103-16-2,4-Benzyloxyphenol and 4-ethoxyphenol,Both
2,4-Benzyloxyphenol and 4-ethoxyphenol,622-62-8,4-Benzyloxyphenol and 4-ethoxyphenol,Both
3,Bithionol,97-18-7,Bithionol (INN),Both
4,Pilocarpine and its salts,92-13-7,Pilocarpine and its salts,Both
...,...,...,...,...
7634,NaN,3830-45-3,Nonadecafluorodecanoic acid [1] Ammonium nonad...,EU
7635,NaN,5625-90-1,"N,N′-Methylenedimorpholine; N,N′-methylenebism...",EU
7636,NaN,55219-65-3,"Triadimenol (ISO); (1RS,2RS;1RS,2SR)-1-(4-chlo...",EU
7637,NaN,111988-49-9,Thiacloprid (ISO); (Z)-3-(6-chloro-3-pyridyl-m...,EU


Just double checking if any error (NaNs):


In [85]:
comparison_nan = comparison[comparison["Exist"].isna()]
len(comparison_nan)

0

### Filter names - EU only, Taiwan only and both as 3 dataframes


In [106]:
comp_eu = (
    comparison.query("Exist == 'EU'").drop_duplicates(keep=False).reset_index(drop=True)
)
comp_eu

,Chemical Name,Exist
0,N-(5-Chlorobenzoxazol-2-yl)acetamide,EU
1,(2-Acetoxyethyl)trimethylammonium hydroxide (A...,EU
2,Aminocaproic acid (INN) and its salts,EU
3,Aconitine (principal alkaloid of Aconitum nape...,EU
4,"Alkyne alcohols, their esters, ethers and salts",EU
...,...,...
1162,Reaction products of paraformaldehyde with 2- ...,EU
1163,Methylhydrazine,EU
1164,"Triadimenol (ISO); (1RS,2RS;1RS,2SR)-1-(4-chlo...",EU
1165,Thiacloprid (ISO); (Z)-3-(6-chloro-3-pyridyl-m...,EU


In [107]:
comp_tw = (
    comparison.query("Exist == 'Taiwan'")
    .drop_duplicates(keep=False)
    .reset_index(drop=True)
)
comp_tw

,Chemical Name,Exist
0,Mercury and its compounds (with the exception ...,Taiwan
1,Bithionol,Taiwan
2,Halogeno-salicylanilide,Taiwan
3,Sodium perborate,Taiwan
4,Sodium borate\nexcept for sodium borate used t...,Taiwan
...,...,...
279,"Phenazinium, 3,7-diamino-2,8-dimethyl-5-phenyl...",Taiwan
280,"Ethanaminium, N-[4-[[4-(diethylamino)phenyl][4...",Taiwan
281,Raw material made from bovine and sheep tissue...,Taiwan
282,Alanroot oil（Inula helenium L.）,Taiwan


In [108]:
comp_both = (
    comparison.query("Exist == 'Both'")
    .drop_duplicates(keep=False)
    .reset_index(drop=True)
)
comp_both

,Chemical Name,Exist
0,Pilocarpine and its salts,Both
1,"Cells, tissues or products of human origin",Both
2,Vinyl chloride monomer,Both
3,"2,2'-Dihydroxy-3,3',5,5',6,6'-hexachlorodiphen...",Both
4,Tretinoin (INN) (retinoic acid and its salts),Both
...,...,...
307,Thallium and its compounds,Both
308,Neodymium and its salts,Both
309,Tellurium and its compounds,Both
310,"Rauwolfia serpentina L., alkaloids and their s...",Both


### Export to excel


Taiwan count no cas no.
EU count no cas no.
Taiwan count cas no.
EU count cas no.

eu can get rid of all the same cas no. item
same with taiwan

eu whats left is the unique items
taiwan whats left is the unique items

in the case of having cas no.:
eu - both = eu unique
taiwan - both = taiwan unique

check whether eu unique is in taiwan unique


In [102]:
with pd.ExcelWriter("./result/25oct_comparison.xlsx") as writer:
    comparison.to_excel(writer, sheet_name="Names Full Comparison")
    comp_eu.to_excel(writer, sheet_name="Only in EU(Names)")
    comp_tw.to_excel(writer, sheet_name="Only in Taiwan(Names)")
    comp_both.to_excel(writer, sheet_name="Exist in Both(Names)")
    cas_comp.to_excel(writer, sheet_name="CAS No. Comparison")
    all_no_cas.to_excel(writer, sheet_name="Everything without CAS No.")
    no_cas_excl.to_excel(writer, sheet_name="Everything with CAS No.")